# Imports

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
sns.set()

# Linear Regression

## Generate linear data with random noise

In [ ]:
X = np.arange(-10, 10.5, 0.5)

Y = 2*X - 3
Y = Y + np.random.normal(0, 1, Y.shape[0])

In [ ]:
X.shape, Y.shape

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("A set of points (x, y)")
plt.scatter(X, Y, c='r')
plt.xlabel('X')
plt.ylabel('Y')
plt.axvline(x=0)
plt.axhline(y=0)
plt.show()

# Line of best fit
1.   Find the line of best fit using the formula of a and b.
2.   Check if you will get the same results with sklearn.
3.   Plot the line alongside the points.



In [ ]:
def compute_line_of_best_fit(X, Y):
  """
  X: 1D np.array of type float representing the input
  Y: 1D np.array of type float representing the label
  return: (a, b) the line parameter
  """
  a = ((X - X.mean())*(Y - Y.mean())).sum()/((X - X.mean())**2).sum()
  b = Y.mean() - a*X.mean()
  return a, b

a_1, b_1 = compute_line_of_best_fit(X, Y)
a_1, b_1

In [ ]:

def compute_line_of_best_fit_with_sklearn(X, Y):
  """
  X: 1D np.array of type float representing the input
  Y: 1D np.array of type float representing the label
  return: (a, b) the line parameter
  """
  reg = LinearRegression().fit(X.reshape(-1, 1), Y.reshape(-1, 1))
  return reg.coef_[0], reg.intercept_[0]

a_2, b_2 = compute_line_of_best_fit_with_sklearn(X, Y)
a_2, b_2

In [ ]:
line_1 = a_1*X + b_1
line_2 = a_2*X + b_2
plt.figure(figsize=(8, 6))
plt.title("A set of points (x, y)")
plt.scatter(X, Y, c='r')
plt.plot(X, line_1, c='b')
plt.plot(X, line_2, c='g')
plt.xlabel('X')
plt.ylabel('Y')
plt.axvline(x=0)
plt.axhline(y=0)
plt.show()

## Generate linear data in high dimensional space

In [ ]:
p, q, n = 70, 50, 100
X = np.random.normal(0, 10, (p, n))
Y = np.random.normal(2, 5, (q, p))@X + np.random.normal(0, 10, (q, 1))

In [ ]:
X.shape, Y.shape

In [ ]:
def compute_normal_equation(X, Y):
  """
  X: 2D np.array of shape (number_of_features, number_of_samples) representing the input
  Y: 2D np.array of shape (number_of_features, number_of_samples) representing the label
  """
  # stack the vectors of ones on top of X
  new_X = np.vstack([np.ones(X.shape[1]), X])
  term_1 = np.dot(Y, new_X.T)
  # np.linalg.inv compute the inverse of a matrix
  term_2 = np.linalg.inv(np.dot(new_X, new_X.T))
  A = np.dot(term_1, term_2)
  return A[:, 1:], A[:, 0]


A_1, b_1 = compute_normal_equation(X, Y)

In [ ]:
def compute_normal_equation_with_sklearn(X, Y):
  """
  X: 2D np.array of shape (number_of_features, number_of_samples) representing the input
  Y: 2D np.array of shape (number_of_features, number_of_samples) representing the label
  """
  reg = LinearRegression().fit(X.T, Y.T)
  return reg.coef_, reg.intercept_

A_2, b_2 = compute_normal_equation_with_sklearn(X, Y)

In [ ]:
np.testing.assert_allclose(A_1, A_2)

In [ ]:
np.testing.assert_allclose(b_1, b_2)

# Logistic Regression

## Generate random data that are linearly separable

In [ ]:
x = np.arange(2, 23, 0.1)

y = 2*x + 10
y_1 = y + np.random.normal(0, 10, y.shape[0])

y = 2*x - 40
y_2 = y + np.random.normal(0, 10, y.shape[0])

X_1 = np.stack([x, y_1])
X_2 = np.stack([x, y_2])
X = np.concatenate([X_1, X_2], axis=1)
Y = np.array([0]*x.shape[0] + [1]*x.shape[0]).reshape(1, -1)

In [ ]:
X.shape, Y.shape

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("A set of points in 2D")
plt.scatter(x, y_1, c='r')
plt.scatter(x, y_2, c='r', marker='x')
plt.xlabel('x')
plt.ylabel('y')
plt.axvline(x=0)
plt.axhline(y=0)

plt.show()

In [ ]:
def sigmoid(x):
  """
  x: np.array of any shape to apply sigmoid elementwise on it.
  return: np.array
  """
  return 1/(1+np.exp(-x))

def logistic_regression(X, a, b):
  """
  X: np.array representing the input to the logistic regression
  a, b: parameters of the logisitc regression
  return: a probability value following the formula of logistic regression
  """
  return sigmoid(np.dot(a, X)+b)

def gradient_a(X, Y, pred):
  """
  Compute the average gradient w.r.t a
  X: input data
  Y: labels
  pred: predictions
  return: the gradient estimate w.r.t a
  """
  return np.mean((-Y + pred)*X, axis=1, keepdims=True).T

def gradient_b(Y, pred):
  """
  Compute the average gradient w.r.t b
  Y: labels
  pred: predictions
  return: the gradient estimate w.r.t b
  """
  return np.mean(-Y + pred)

In [ ]:
def gradient_descent(X, Y, initial_a, initial_b, step):
  a, b = initial_a, initial_b

  pred = logistic_regression(X, a, b)
  grad_a = gradient_a(X, Y, pred)
  grad_b = gradient_b(Y, pred)

  count = 0
  while(np.mean(grad_a**2) > 0.0001 or np.abs(grad_b) > 0.0001):
    a = a - step*grad_a
    b = b - step*grad_b

    pred = logistic_regression(X, a, b)
    grad_a = gradient_a(X, Y, pred)
    grad_b = gradient_b(Y, pred)
    count +=1
    if count > 1e6:
      print("algorithm diverged!")
      break
  return a, b, count


def momentum_gradient_descent(X, Y, initial_a, initial_b, step, alpha):
  a, b = initial_a, initial_b

  v_a = np.zeros_like(initial_a)
  v_b = 0
  pred = logistic_regression(X, a, b)
  grad_a = gradient_a(X, Y, pred)
  grad_b = gradient_b(Y, pred)
  count = 0

  while(np.mean(grad_a**2) > 0.0001 or np.abs(grad_b) > 0.0001):
    v_a = alpha*v_a - step*grad_a
    v_b = alpha*v_b - step*grad_b
    a = a + v_a
    b = b + v_b

    pred = logistic_regression(X, a, b)
    grad_a = gradient_a(X, Y, pred)
    grad_b = gradient_b(Y, pred)
    count +=1
    if count > 1e6:
      print("algorithm diverged!")
      break

  return a, b, count


step = 0.1
alpha = 0.9999
initial_a, initial_b = np.random.normal(scale=0.1, size=(1, 2)), 0
# a_1, b_1, count_1 = gradient_descent(X, Y, initial_a, initial_b, step)
a_2, b_2, count_2 = momentum_gradient_descent(X, Y, initial_a, initial_b, step, alpha)

In [ ]:
count_1, count_2

In [ ]:
def logistic_regression_with_sklearn(X, Y):
  clf = LogisticRegression(penalty=None).fit(X.T, Y.T)
  return clf.coef_, clf.intercept_

a_3, b_3 = logistic_regression_with_sklearn(X, Y)

In [ ]:
line_1 = -a_1[0][0]*x/a_1[0][1] -b_1/a_1[0][1]
line_2 = -a_3[0][0]*x/a_3[0][1] -b_3/a_3[0][1]

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("A set of points in 2D")
plt.scatter(x, y_1, c='r')
plt.scatter(x, y_2, c='r', marker='x')
plt.plot(x, line_1, c='y')
plt.plot(x, line_2, c='g')
plt.xlabel('x')
plt.ylabel('y')
plt.axvline(x=0)
plt.axhline(y=0)

plt.show()